# Using tf-shell with Multiple Machines

`tf-shell` can be run on multiple machines using TensorFlow device placement.

A TensorFlow cluster is set up by each machine running something like the
following:

```python
cluster = tf.train.ClusterSpec('''{
  "alice": ["alice.com:2222"],
  "bob": ["bob.com:2223"],
}''')

server = tf.distribute.Server(
    cluster,
    job_name="/job:alice/replica:0/task:0/device:CPU:0",  # or bob
    task_index=0,
)

tf.config.experimental_connect_to_cluster(cluster)
```

In this notebook, we will emulate distributed execution on a single machine by
using the special job name `/job:localhost/replica:0/task:0/device:CPU:0` for
both alice and bob, and skip the server setup.

In [1]:
alice = "/job:localhost/replica:0/task:0/device:CPU:0"
bob = "/job:localhost/replica:0/task:0/device:CPU:0"

Since `tf-shell` works with sensitive cryptographic material, it is important to
tell TensorFlow to only place ops on devices which were explicitly assigned,
for security reasons.

In [2]:
import tensorflow as tf
tf.config.set_soft_device_placement(False)

2024-10-30 05:37:22.799257: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-30 05:37:22.825891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


TensorFlow makes it easy to schedule operations on specific parties. In this
example, Alice will generate a secret key, encrypt the input x, and send it to
Bob. Bob will square the value, and return to alice, who will decrypt it.

In [3]:
import tf_shell

param_cache = "/tmp/example_protocol_param_cache"

@tf.function
def example_protocol(x):
  with tf.device(alice):
    shell_context = tf_shell.create_autocontext64(
        log2_cleartext_sz=6,
        scaling_factor=1,
        noise_offset_log2=0,
        cache_path=param_cache,
    )
    key = tf_shell.create_key64(shell_context, param_cache)

    enc_x = tf_shell.to_encrypted(x, key, shell_context)

  with tf.device(bob):
    enc_x_squared = enc_x * enc_x
  
  with tf.device(alice):
    x_squared = tf_shell.to_tensorflow(enc_x_squared, key)
    return x_squared

# Turn on shell graph optimizers and deferred execution to use autocontext.
tf_shell.enable_optimization()
tf.config.run_functions_eagerly(False)

res = example_protocol(tf.constant([5.0]))
print(res[0])

Selected BGV parameters:
log_n: 11
t: 12289 (14 bits, min:6)
qs: 2251800363651073  (52 bits, min:52 = t:14 + noise:38 + offset:0)
INFO: Generating key
tf.Tensor(25.0, shape=(), dtype=float32)


In this example, we used the `cache_path` arguments when creating the context
and key. This prevents regenerating the parameters every time the code is run.
If we call the function again, these parameters will be loaded from the cache.

In [4]:
res = example_protocol(tf.constant([6.0]))
print(res[0])

INFO: Generating key
tf.Tensor(36.0, shape=(), dtype=float32)
